In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import statsmodels.api as sm
from statsmodels.formula.api import ols
np.NaN = np.nan
import pandas_ta as ta


In [3]:
df = pd.read_csv('/home/sacsresta/Documents/RESEARCH/Project/sentiment_categorical/merged_data_AAPL_from_2015-01-01_to_2025-03-01.csv')

In [4]:
df.drop(columns=['Ticker','headline'], inplace = True)

In [59]:
df

,Date,Adj Close,Close,High,Low,Open,Volume,sentiment_score,sentiment_class
0,2015-01-02,24.320433,27.332500,27.860001,26.837500,27.847500,212818400,-0.357845,positive
1,2015-01-05,23.635286,26.562500,27.162500,26.352501,27.072500,257142000,-0.017792,negative
2,2015-01-06,23.637514,26.565001,26.857500,26.157499,26.635000,263188400,0.773892,neutral
3,2015-01-07,23.968966,26.937500,27.049999,26.674999,26.799999,160423600,0.227357,negative
4,2015-01-08,24.889906,27.972500,28.037500,27.174999,27.307501,237458000,0.402678,neutral
...,...,...,...,...,...,...,...,...,...
2533,2025-02-24,247.100006,247.100006,248.860001,244.419998,244.929993,51326400,-0.528224,positive
2534,2025-02-25,247.039993,247.039993,250.000000,244.910004,248.000000,48013300,-0.234334,negative
2535,2025-02-26,240.360001,240.360001,244.979996,239.130005,244.330002,44433600,0.336853,negative
2536,2025-02-27,237.300003,237.300003,242.460007,237.059998,239.410004,41153600,-0.126740,negative


In [139]:
def SSL(data, period):
    data['smaHigh'] = data['High'].rolling(window=period).mean()
    data['smaLow'] = data['Low'].rolling(window=period).mean()
    data['sslDown'] = data.apply(lambda row: row['smaHigh'] if row['Close'] < row['smaLow'] else row['smaLow'], axis=1)
    data['sslUp'] = data.apply(lambda row: row['smaLow'] if row['Close'] < row['smaLow'] else row['smaHigh'], axis=1)
    data['Trend'] = data.apply(lambda row: "buy" if row['smaLow'] == row['sslDown'] else "sell", axis=1)
    data['ssl_signal'] = (data['Trend'] != data['Trend'].shift(1)).astype(int)
    return data

In [140]:
ssl_df = SSL(data=df.copy(), period=10)
ssl_df.dropna(inplace=True)

In [141]:
ssl_df.reset_index(drop=True, inplace=True)

In [142]:
ssl_df

,Date,Adj Close,Close,High,Low,Open,Volume,sentiment_score,sentiment_class,smaHigh,smaLow,sslDown,sslUp,Trend,ssl_signal
0,2015-01-15,23.762087,26.705000,27.514999,26.665001,27.500000,240056000,0.070656,negative,27.677500,26.896750,27.677500,26.896750,sell,0
1,2015-01-16,23.577454,26.497499,26.895000,26.299999,26.757500,314053200,-0.808065,positive,27.581000,26.843000,27.581000,26.843000,sell,0
2,2015-01-20,24.184740,27.180000,27.242500,26.625000,26.959999,199599600,0.814068,neutral,27.589000,26.870250,26.870250,27.589000,buy,1
3,2015-01-21,24.369370,27.387501,27.764999,27.067499,27.237499,194303600,0.017530,negative,27.679750,26.961250,26.961250,27.679750,buy,0
4,2015-01-22,25.003349,28.100000,28.117500,27.430000,27.565001,215185600,0.770328,neutral,27.786500,27.036750,27.036750,27.786500,buy,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2524,2025-02-24,247.100006,247.100006,248.860001,244.419998,244.929993,51326400,-0.528224,positive,242.618999,238.150000,238.150000,242.618999,buy,0
2525,2025-02-25,247.039993,247.039993,250.000000,244.910004,248.000000,48013300,-0.234334,negative,244.559999,239.921001,239.921001,244.559999,buy,0
2526,2025-02-26,240.360001,240.360001,244.979996,239.130005,244.330002,44433600,0.336853,negative,245.534999,241.021001,245.534999,241.021001,sell,1
2527,2025-02-27,237.300003,237.300003,242.460007,237.059998,239.410004,41153600,-0.126740,negative,246.084999,241.659001,246.084999,241.659001,sell,0


In [143]:
def Signal_rule_sentiment(row):
    if row['ssl_signal'] == 1 and row['sentiment_class'] == 'positive' and row['Trend'] == 'buy':
        return 1
    elif row['ssl_signal'] == 1 and row['sentiment_class'] == 'negative' and row['Trend'] == 'sell':
        return -1
    else:
        return 0

# Apply the function row-wise
ssl_df['combined_signal'] = ssl_df.apply(Signal_rule_sentiment, axis=1)

In [144]:
ssl_df['combined_signal'].value_counts()

combined_signal
 0    2394
-1      89
 1      46
Name: count, dtype: int64

In [145]:
def get_ssl_signal():
    return ssl_df['ssl_signal'].values

def get_trend():
    return ssl_df['Trend'].values

def get_news_class():
    return ssl_df['sentiment_class'].values

def get_combined_signal():
    return ssl_df.combined_signal.values

In [146]:
get_combined_signal()

array([ 0,  0,  0, ..., -1,  0,  0])

In [147]:
ssl_df.dtypes

Date                object
Adj Close          float64
Close              float64
High               float64
Low                float64
Open               float64
Volume               int64
sentiment_score    float64
sentiment_class     object
smaHigh            float64
smaLow             float64
sslDown            float64
sslUp              float64
Trend               object
ssl_signal           int64
combined_signal      int64
dtype: object

In [148]:
data= ssl_df[['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume',
       'sentiment_score',  'smaHigh', 'smaLow', 'sslDown',
       'sslUp', 'ssl_signal', 'combined_signal']].copy(
)

In [149]:
data

,Adj Close,Close,High,Low,Open,Volume,sentiment_score,smaHigh,smaLow,sslDown,sslUp,ssl_signal,combined_signal
0,23.762087,26.705000,27.514999,26.665001,27.500000,240056000,0.070656,27.677500,26.896750,27.677500,26.896750,0,0
1,23.577454,26.497499,26.895000,26.299999,26.757500,314053200,-0.808065,27.581000,26.843000,27.581000,26.843000,0,0
2,24.184740,27.180000,27.242500,26.625000,26.959999,199599600,0.814068,27.589000,26.870250,26.870250,27.589000,1,0
3,24.369370,27.387501,27.764999,27.067499,27.237499,194303600,0.017530,27.679750,26.961250,26.961250,27.679750,0,0
4,25.003349,28.100000,28.117500,27.430000,27.565001,215185600,0.770328,27.786500,27.036750,27.036750,27.786500,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2524,247.100006,247.100006,248.860001,244.419998,244.929993,51326400,-0.528224,242.618999,238.150000,238.150000,242.618999,0,0
2525,247.039993,247.039993,250.000000,244.910004,248.000000,48013300,-0.234334,244.559999,239.921001,239.921001,244.559999,0,0
2526,240.360001,240.360001,244.979996,239.130005,244.330002,44433600,0.336853,245.534999,241.021001,245.534999,241.021001,1,-1
2527,237.300003,237.300003,242.460007,237.059998,239.410004,41153600,-0.126740,246.084999,241.659001,246.084999,241.659001,0,0


In [150]:
data.reset_index(inplace=True)

In [151]:
data.index = pd.to_datetime(data.index)

In [161]:
ssl_df.set_index('Date',inplace=True)

In [166]:
ssl_df.index = pd.to_datetime(ssl_df.index)

In [167]:
ssl_df.index.dtype

dtype('<M8[ns]')

In [170]:
ssl_df

,Adj Close,Close,High,Low,Open,Volume,sentiment_score,sentiment_class,smaHigh,smaLow,sslDown,sslUp,Trend,ssl_signal,combined_signal
Date,,,,,,,,,,,,,,,
2015-01-15,23.762087,26.705000,27.514999,26.665001,27.500000,240056000,0.070656,negative,27.677500,26.896750,27.677500,26.896750,sell,0,0
2015-01-16,23.577454,26.497499,26.895000,26.299999,26.757500,314053200,-0.808065,positive,27.581000,26.843000,27.581000,26.843000,sell,0,0
2015-01-20,24.184740,27.180000,27.242500,26.625000,26.959999,199599600,0.814068,neutral,27.589000,26.870250,26.870250,27.589000,buy,1,0
2015-01-21,24.369370,27.387501,27.764999,27.067499,27.237499,194303600,0.017530,negative,27.679750,26.961250,26.961250,27.679750,buy,0,0
2015-01-22,25.003349,28.100000,28.117500,27.430000,27.565001,215185600,0.770328,neutral,27.786500,27.036750,27.036750,27.786500,buy,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-24,247.100006,247.100006,248.860001,244.419998,244.929993,51326400,-0.528224,positive,242.618999,238.150000,238.150000,242.618999,buy,0,0
2025-02-25,247.039993,247.039993,250.000000,244.910004,248.000000,48013300,-0.234334,negative,244.559999,239.921001,239.921001,244.559999,buy,0,0
2025-02-26,240.360001,240.360001,244.979996,239.130005,244.330002,44433600,0.336853,negative,245.534999,241.021001,245.534999,241.021001,sell,1,-1


In [182]:
ssl_df.ssl_signal.value_counts()

ssl_signal
0    2160
1     369
Name: count, dtype: int64

In [186]:
ssl_df.ssl_signal.shift(-1).dropna().values

array([0., 1., 0., ..., 1., 0., 1.])

In [192]:
ssl_df.iloc[:]

,Adj Close,Close,High,Low,Open,Volume,sentiment_score,sentiment_class,smaHigh,smaLow,sslDown,sslUp,Trend,ssl_signal,combined_signal
Date,,,,,,,,,,,,,,,
2015-01-15,23.762087,26.705000,27.514999,26.665001,27.500000,240056000,0.070656,negative,27.677500,26.896750,27.677500,26.896750,sell,0,0
2015-01-16,23.577454,26.497499,26.895000,26.299999,26.757500,314053200,-0.808065,positive,27.581000,26.843000,27.581000,26.843000,sell,0,0
2015-01-20,24.184740,27.180000,27.242500,26.625000,26.959999,199599600,0.814068,neutral,27.589000,26.870250,26.870250,27.589000,buy,1,0
2015-01-21,24.369370,27.387501,27.764999,27.067499,27.237499,194303600,0.017530,negative,27.679750,26.961250,26.961250,27.679750,buy,0,0
2015-01-22,25.003349,28.100000,28.117500,27.430000,27.565001,215185600,0.770328,neutral,27.786500,27.036750,27.036750,27.786500,buy,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-24,247.100006,247.100006,248.860001,244.419998,244.929993,51326400,-0.528224,positive,242.618999,238.150000,238.150000,242.618999,buy,0,0
2025-02-25,247.039993,247.039993,250.000000,244.910004,248.000000,48013300,-0.234334,negative,244.559999,239.921001,239.921001,244.559999,buy,0,0
2025-02-26,240.360001,240.360001,244.979996,239.130005,244.330002,44433600,0.336853,negative,245.534999,241.021001,245.534999,241.021001,sell,1,-1


In [196]:
from backtesting import Strategy, Backtest

def get_signal():
    return ssl_df['ssl_signal'].values # Ensure it's a proper numeric Series

class MyStrategy(Strategy):
    def init(self):
        self.signal = self.I(get_signal)
        self.trade_log = []  

    def next(self):
        if self.signal == 1:
            
            if not self.position:
                self.buy()
                self.trade_log.append({
                    'action': 'BUY',
                    'price': self.data.Close[-1],
                    'signal': self.signal[-1],
                    'date': self.data.index[-1]
                })

        elif self.signal == 0:
            
            
            if self.position:
                self.position.close()
                self.trade_log.append({
                    'action': 'SELL',
                    'price': self.data.Close[-1],
                    'signal': self.signal[-1],
                    'date': self.data.index[-1]
                })

        

    def analyze_custom_log(self):
        return pd.DataFrame(self.trade_log)

bt = Backtest(ssl_df, MyStrategy, cash=100000)
stats = bt.run()
stats


Backtest.run:   0%|          | 0/2528 [00:00<?, ?it/s]

Start                     2015-01-15 00:00:00
End                       2025-02-28 00:00:00
Duration                   3697 days 00:00:00
Exposure Time [%]                    25.02966
Equity Final [$]                  78415.80007
Equity Peak [$]                  131310.31173
Return [%]                           -21.5842
Buy & Hold Return [%]               805.59819
Return (Ann.) [%]                    -2.39368
Volatility (Ann.) [%]                12.86635
CAGR [%]                              -1.6437
Sharpe Ratio                         -0.18604
Sortino Ratio                        -0.23826
Calmar Ratio                         -0.05646
Alpha [%]                          -192.62401
Beta                                  0.21231
Max. Drawdown [%]                   -42.39458
Avg. Drawdown [%]                    -5.18266
Max. Drawdown Duration     2311 days 00:00:00
Avg. Drawdown Duration      211 days 00:00:00
# Trades                                  265
Win Rate [%]                      

In [197]:
ssl_df['shifted_ssl_signal'] = ssl_df['ssl_signal'].shift(-1)

In [198]:
shifted_df = ssl_df.dropna()

In [199]:
shifted_df

,Adj Close,Close,High,Low,Open,Volume,sentiment_score,sentiment_class,smaHigh,smaLow,sslDown,sslUp,Trend,ssl_signal,combined_signal,shifted_ssl_signal
Date,,,,,,,,,,,,,,,,
2015-01-15,23.762087,26.705000,27.514999,26.665001,27.500000,240056000,0.070656,negative,27.677500,26.896750,27.677500,26.896750,sell,0,0,0.0
2015-01-16,23.577454,26.497499,26.895000,26.299999,26.757500,314053200,-0.808065,positive,27.581000,26.843000,27.581000,26.843000,sell,0,0,1.0
2015-01-20,24.184740,27.180000,27.242500,26.625000,26.959999,199599600,0.814068,neutral,27.589000,26.870250,26.870250,27.589000,buy,1,0,0.0
2015-01-21,24.369370,27.387501,27.764999,27.067499,27.237499,194303600,0.017530,negative,27.679750,26.961250,26.961250,27.679750,buy,0,0,0.0
2015-01-22,25.003349,28.100000,28.117500,27.430000,27.565001,215185600,0.770328,neutral,27.786500,27.036750,27.036750,27.786500,buy,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-21,245.550003,245.550003,248.690002,245.220001,245.949997,53197400,0.060861,neutral,241.132999,236.434000,236.434000,241.132999,buy,0,0,0.0
2025-02-24,247.100006,247.100006,248.860001,244.419998,244.929993,51326400,-0.528224,positive,242.618999,238.150000,238.150000,242.618999,buy,0,0,0.0
2025-02-25,247.039993,247.039993,250.000000,244.910004,248.000000,48013300,-0.234334,negative,244.559999,239.921001,239.921001,244.559999,buy,0,0,1.0


In [200]:
from backtesting import Strategy, Backtest

def get_signal():
    return shifted_df['shifted_ssl_signal'].values # Ensure it's a proper numeric Series

class MyStrategy(Strategy):
    def init(self):
        self.signal = self.I(get_signal)
        self.trade_log = []  

    def next(self):
        if self.signal == 1:
            
            if not self.position:
                self.buy()
                self.trade_log.append({
                    'action': 'BUY',
                    'price': self.data.Close[-1],
                    'signal': self.signal[-1],
                    'date': self.data.index[-1]
                })

        elif self.signal == 0:
            
            
            if self.position:
                self.position.close()
                self.trade_log.append({
                    'action': 'SELL',
                    'price': self.data.Close[-1],
                    'signal': self.signal[-1],
                    'date': self.data.index[-1]
                })

        

    def analyze_custom_log(self):
        return pd.DataFrame(self.trade_log)

bt = Backtest(shifted_df, MyStrategy, cash=100000)
stats = bt.run()
stats


Backtest.run:   0%|          | 0/2527 [00:00<?, ?it/s]

Start                     2015-01-15 00:00:00
End                       2025-02-27 00:00:00
Duration                   3696 days 00:00:00
Exposure Time [%]                    25.03956
Equity Final [$]                  53879.56373
Equity Peak [$]                  106805.39847
Return [%]                          -46.12044
Buy & Hold Return [%]               788.59765
Return (Ann.) [%]                    -5.97845
Volatility (Ann.) [%]                13.68281
CAGR [%]                             -4.12884
Sharpe Ratio                         -0.43693
Sortino Ratio                        -0.54983
Calmar Ratio                         -0.11398
Alpha [%]                          -273.15982
Beta                                   0.2879
Max. Drawdown [%]                   -52.45144
Avg. Drawdown [%]                   -19.22573
Max. Drawdown Duration     3489 days 00:00:00
Avg. Drawdown Duration     1207 days 00:00:00
# Trades                                  265
Win Rate [%]                      

In [157]:
stats = bt.run()

Backtest.run:   0%|          | 0/2528 [00:00<?, ?it/s]

In [158]:
stats

Start                     1970-01-01 00:00:00
End                       1970-01-01 00:00...
Duration                  0 days 00:00:00....
Exposure Time [%]                         0.0
Equity Final [$]                 870393.83187
Equity Peak [$]                  932224.62551
Return [%]                          770.39383
Buy & Hold Return [%]               805.59819
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
CAGR [%]                                  NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Alpha [%]                           -34.33888
Beta                                  0.99893
Max. Drawdown [%]                   -38.72753
Avg. Drawdown [%]                    -4.64657
Max. Drawdown Duration    0 days 00:00:00....
Avg. Drawdown Duration    0 days 00:00:00....
# Trades                                    0
Win Rate [%]                      

In [217]:
df.drop(columns=['index'],inplace=True)

In [218]:
df

,Date,Adj Close,Close,High,Low,Open,Volume,sentiment_score,sentiment_class
0,2015-01-02,24.320433,27.332500,27.860001,26.837500,27.847500,212818400,-0.357845,positive
1,2015-01-05,23.635286,26.562500,27.162500,26.352501,27.072500,257142000,-0.017792,negative
2,2015-01-06,23.637514,26.565001,26.857500,26.157499,26.635000,263188400,0.773892,neutral
3,2015-01-07,23.968966,26.937500,27.049999,26.674999,26.799999,160423600,0.227357,negative
4,2015-01-08,24.889906,27.972500,28.037500,27.174999,27.307501,237458000,0.402678,neutral
...,...,...,...,...,...,...,...,...,...
2533,2025-02-24,247.100006,247.100006,248.860001,244.419998,244.929993,51326400,-0.528224,positive
2534,2025-02-25,247.039993,247.039993,250.000000,244.910004,248.000000,48013300,-0.234334,negative
2535,2025-02-26,240.360001,240.360001,244.979996,239.130005,244.330002,44433600,0.336853,negative
2536,2025-02-27,237.300003,237.300003,242.460007,237.059998,239.410004,41153600,-0.126740,negative


In [219]:
df.dtypes

Date               datetime64[ns]
Adj Close                 float64
Close                     float64
High                      float64
Low                       float64
Open                      float64
Volume                      int64
sentiment_score           float64
sentiment_class            object
dtype: object

In [207]:
df.Date = pd.to_datetime(df.Date)

In [222]:
df = df.sort_index()

In [224]:
df.set_index('Date', inplace=True)  # Set it as index
df

,Adj Close,Close,High,Low,Open,Volume,sentiment_score,sentiment_class
Date,,,,,,,,
2015-01-02,24.320433,27.332500,27.860001,26.837500,27.847500,212818400,-0.357845,positive
2015-01-05,23.635286,26.562500,27.162500,26.352501,27.072500,257142000,-0.017792,negative
2015-01-06,23.637514,26.565001,26.857500,26.157499,26.635000,263188400,0.773892,neutral
2015-01-07,23.968966,26.937500,27.049999,26.674999,26.799999,160423600,0.227357,negative
2015-01-08,24.889906,27.972500,28.037500,27.174999,27.307501,237458000,0.402678,neutral
...,...,...,...,...,...,...,...,...
2025-02-24,247.100006,247.100006,248.860001,244.419998,244.929993,51326400,-0.528224,positive
2025-02-25,247.039993,247.039993,250.000000,244.910004,248.000000,48013300,-0.234334,negative
2025-02-26,240.360001,240.360001,244.979996,239.130005,244.330002,44433600,0.336853,negative


In [225]:
df['vwap'] = ta.vwap(high=df.High, low=df.Low, close=df.Close, volume=df.Volume)

In [226]:
df

,Adj Close,Close,High,Low,Open,Volume,sentiment_score,sentiment_class,vwap
Date,,,,,,,,,
2015-01-02,24.320433,27.332500,27.860001,26.837500,27.847500,212818400,-0.357845,positive,27.343334
2015-01-05,23.635286,26.562500,27.162500,26.352501,27.072500,257142000,-0.017792,negative,26.692500
2015-01-06,23.637514,26.565001,26.857500,26.157499,26.635000,263188400,0.773892,neutral,26.526667
2015-01-07,23.968966,26.937500,27.049999,26.674999,26.799999,160423600,0.227357,negative,26.887499
2015-01-08,24.889906,27.972500,28.037500,27.174999,27.307501,237458000,0.402678,neutral,27.728333
...,...,...,...,...,...,...,...,...,...
2025-02-24,247.100006,247.100006,248.860001,244.419998,244.929993,51326400,-0.528224,positive,246.793335
2025-02-25,247.039993,247.039993,250.000000,244.910004,248.000000,48013300,-0.234334,negative,247.316666
2025-02-26,240.360001,240.360001,244.979996,239.130005,244.330002,44433600,0.336853,negative,241.490000


In [227]:
df['vwap_signal'] = np.where(
    df['Close'] > df['vwap'], 1,    # Price above VWAP - Bullish
    -1                              # Price below VWAP - Bearish
)

In [234]:
df['vwap_signal_shifted'] = df['vwap_signal'].shift(-1)
df = df.dropna()

In [238]:
df

,Adj Close,Close,High,Low,Open,Volume,sentiment_score,sentiment_class,vwap,vwap_signal,vwap_signal_shifted
Date,,,,,,,,,,,
2015-01-02,24.320433,27.332500,27.860001,26.837500,27.847500,212818400,-0.357845,positive,27.343334,-1,-1.0
2015-01-05,23.635286,26.562500,27.162500,26.352501,27.072500,257142000,-0.017792,negative,26.692500,-1,1.0
2015-01-06,23.637514,26.565001,26.857500,26.157499,26.635000,263188400,0.773892,neutral,26.526667,1,1.0
2015-01-07,23.968966,26.937500,27.049999,26.674999,26.799999,160423600,0.227357,negative,26.887499,1,1.0
2015-01-08,24.889906,27.972500,28.037500,27.174999,27.307501,237458000,0.402678,neutral,27.728333,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...
2025-02-21,245.550003,245.550003,248.690002,245.220001,245.949997,53197400,0.060861,neutral,246.486669,-1,1.0
2025-02-24,247.100006,247.100006,248.860001,244.419998,244.929993,51326400,-0.528224,positive,246.793335,1,-1.0
2025-02-25,247.039993,247.039993,250.000000,244.910004,248.000000,48013300,-0.234334,negative,247.316666,-1,-1.0


In [245]:
from backtesting import Strategy, Backtest

def get_signal():
    return df['vwap_signal'].iloc[200:600].values # Ensure it's a proper numeric Series

class MyStrategy(Strategy):
    def init(self):
        self.signal = self.I(get_signal)
        self.trade_log = []  

    def next(self):
        if self.signal == 1:
            
            if not self.position:
                self.buy()
                self.trade_log.append({
                    'action': 'BUY',
                    'price': self.data.Close[-1],
                    'signal': self.signal[-1],
                    'date': self.data.index[-1]
                })

        elif self.signal == -1:
            
            
            if self.position:
                self.position.close()
                self.trade_log.append({
                    'action': 'SELL',
                    'price': self.data.Close[-1],
                    'signal': self.signal[-1],
                    'date': self.data.index[-1]
                })

        

    def analyze_custom_log(self):
        return pd.DataFrame(self.trade_log)

bt = Backtest(df.iloc[200:600], MyStrategy, cash=100000)
stats = bt.run()
stats


Backtest.run:   0%|          | 0/399 [00:00<?, ?it/s]

Start                     2015-10-19 00:00:00
End                       2017-05-23 00:00:00
Duration                    582 days 00:00:00
Exposure Time [%]                        80.0
Equity Final [$]                  98107.54327
Equity Peak [$]                   104003.1903
Return [%]                           -1.89246
Buy & Hold Return [%]                37.65327
Return (Ann.) [%]                    -1.19646
Volatility (Ann.) [%]                14.53223
CAGR [%]                             -0.82385
Sharpe Ratio                         -0.08233
Sortino Ratio                        -0.11777
Calmar Ratio                         -0.04404
Alpha [%]                           -19.14293
Beta                                  0.45814
Max. Drawdown [%]                    -27.1645
Avg. Drawdown [%]                    -8.05421
Max. Drawdown Duration      550 days 00:00:00
Avg. Drawdown Duration      145 days 00:00:00
# Trades                                  108
Win Rate [%]                      

In [247]:
scaled_data = df[['Close','sentiment_score','vwap_signal_shifted']]

In [248]:
train_size = int(len(scaled_data) * 0.8)
train_data, test_data = scaled_data[0:train_size], scaled_data[train_size:]

In [249]:
train_data

,Close,sentiment_score,vwap_signal_shifted
Date,,,
2015-01-02,27.332500,-0.357845,-1.0
2015-01-05,26.562500,-0.017792,1.0
2015-01-06,26.565001,0.773892,1.0
2015-01-07,26.937500,0.227357,1.0
2015-01-08,27.972500,0.402678,1.0
...,...,...,...
2023-02-10,151.009995,0.003720,1.0
2023-02-13,153.850006,-0.129063,1.0
2023-02-14,153.199997,0.098157,1.0


In [250]:
test_data

,Close,sentiment_score,vwap_signal_shifted
Date,,,
2023-02-17,152.550003,0.068906,-1.0
2023-02-21,148.479996,0.134218,1.0
2023-02-22,148.910004,0.010898,1.0
2023-02-23,149.399994,0.136820,1.0
2023-02-24,146.710007,0.676635,-1.0
...,...,...,...
2025-02-21,245.550003,0.060861,1.0
2025-02-24,247.100006,-0.528224,-1.0
2025-02-25,247.039993,-0.234334,-1.0


In [302]:
import numpy as np

def create_sequences(data, look_back=4):
    X, y = [], []
    for i in range(len(data) - look_back - 1):
        X.append(data.iloc[i:(i + look_back), :2])  
        y.append(data.iloc[(i + look_back), 2])      # Predict 'Close' (column 0)
    return np.array(X), np.array(y)

look_back = 4  # Adjust based on experimentation
X_train, y_train = create_sequences(train_data, look_back)
X_test, y_test = create_sequences(test_data, look_back)

In [306]:
X_train[0]

array([[ 2.73325005e+01, -3.57844900e-01],
       [ 2.65625000e+01, -1.77923660e-02],
       [ 2.65650005e+01,  7.73892100e-01],
       [ 2.69375000e+01,  2.27356930e-01]])

In [305]:
y_train.shape

(2024,)

In [314]:
train_data.iloc[0:5,:2].iloc[-1,-1]

np.float64(0.40267763)

In [295]:
train_data[61,2]

KeyError: (61, 2)

In [277]:
y_train

array([[ 2.79724998e+01,  4.02677630e-01],
       [ 2.80025005e+01,  1.64295140e-02],
       [ 2.73125000e+01,  4.35749020e-01],
       ...,
       [ 1.53850006e+02, -1.29063070e-01],
       [ 1.53199997e+02,  9.81572100e-02],
       [ 1.55330002e+02, -3.88089270e-02]])

In [276]:
X_train

array([[[ 2.73325005e+01, -3.57844900e-01],
        [ 2.65625000e+01, -1.77923660e-02],
        [ 2.65650005e+01,  7.73892100e-01],
        [ 2.69375000e+01,  2.27356930e-01]],

       [[ 2.65625000e+01, -1.77923660e-02],
        [ 2.65650005e+01,  7.73892100e-01],
        [ 2.69375000e+01,  2.27356930e-01],
        [ 2.79724998e+01,  4.02677630e-01]],

       [[ 2.65650005e+01,  7.73892100e-01],
        [ 2.69375000e+01,  2.27356930e-01],
        [ 2.79724998e+01,  4.02677630e-01],
        [ 2.80025005e+01,  1.64295140e-02]],

       ...,

       [[ 1.54649994e+02,  7.30652630e-01],
        [ 1.51919998e+02,  5.92209800e-01],
        [ 1.50869995e+02, -4.11608300e-01],
        [ 1.51009995e+02,  3.71984400e-03]],

       [[ 1.51919998e+02,  5.92209800e-01],
        [ 1.50869995e+02, -4.11608300e-01],
        [ 1.51009995e+02,  3.71984400e-03],
        [ 1.53850006e+02, -1.29063070e-01]],

       [[ 1.50869995e+02, -4.11608300e-01],
        [ 1.51009995e+02,  3.71984400e-03],
        [